In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from utils.model_val_metrics_helper import select_threshold, metric_null, plot_intervention_counts

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
pd.options.display.float_format = '{:,.4f}'.format

# I. Combine Main DF with Intervention DF

In [2]:
pred_df_full = pd.read_pickle('data/pred_df_full_20220805_20220930_20221010.pkl')
pred_df_full.iloc[:, 3:]

,DepartmentServiceGrouper,DepartmentName,LocationAbbreviation,LOS,rel_time,abs_time,AdmissionInstant,AdmissionDateKey,etco2,SBP,DBP,HR,Resp,Temp,Weight,BMI,SpO2,gcs_total_score,MAP,ALKALINE PHOSPHATASE,APTT,"ARTERIAL PCO2, POC","ARTERIAL PH, POC","ARTERIAL PO2, POC",AST,BILIRUBIN TOTAL,BLOOD UREA NITROGEN,CALCIUM,CHLORIDE,CREATININE,FIBRINOGEN,"FIO2, ABG",GLUCOSE,HEMATOCRIT,HEMOGLOBIN,"LACTATE, ABG",MAGNESIUM,"OXYGEN PERCENT, ABG","PCO2, ABG","PH, ABG","PHOSPHORUS, INORGANIC",PLATELET COUNT,"PO2, ABG",POTASSIUM,SODIUM,WBC,AgeAtAdmissioninYears,sex_Female,sex_Male,etco2_interval_f1,etco2_interval_f2,etco2_diff,SBP_interval_f1,SBP_interval_f2,SBP_diff,DBP_interval_f1,DBP_interval_f2,DBP_diff,HR_interval_f1,HR_interval_f2,HR_diff,Resp_interval_f1,Resp_interval_f2,Resp_diff,Temp_interval_f1,Temp_interval_f2,Temp_diff,Weight_interval_f1,Weight_interval_f2,Weight_diff,BMI_interval_f1,BMI_interval_f2,BMI_diff,SpO2_interval_f1,SpO2_interval_f2,SpO2_diff,gcs_total_score_interval_f1,gcs_total_score_interval_f2,gcs_total_score_diff,MAP_interval_f1,MAP_interval_f2,MAP_diff,ALKALINE PHOSPHATASE_interval_f1,ALKALINE PHOSPHATASE_interval_f2,ALKALINE PHOSPHATASE_diff,APTT_interval_f1,APTT_interval_f2,APTT_diff,"ARTERIAL PCO2, POC_interval_f1","ARTERIAL PCO2, POC_interval_f2","ARTERIAL PCO2, POC_diff","ARTERIAL PH, POC_interval_f1","ARTERIAL PH, POC_interval_f2","ARTERIAL PH, POC_diff","ARTERIAL PO2, POC_interval_f1","ARTERIAL PO2, POC_interval_f2","ARTERIAL PO2, POC_diff",AST_interval_f1,AST_interval_f2,AST_diff,BILIRUBIN TOTAL_interval_f1,BILIRUBIN TOTAL_interval_f2,BILIRUBIN TOTAL_diff,BLOOD UREA NITROGEN_interval_f1,BLOOD UREA NITROGEN_interval_f2,BLOOD UREA NITROGEN_diff,CALCIUM_interval_f1,CALCIUM_interval_f2,CALCIUM_diff,CHLORIDE_interval_f1,CHLORIDE_interval_f2,CHLORIDE_diff,CREATININE_interval_f1,CREATININE_interval_f2,CREATININE_diff,"FIO2, ABG_interval_f1","FIO2, ABG_interval_f2","FIO2, ABG_diff",GLUCOSE_interval_f1,GLUCOSE_interval_f2,GLUCOSE_diff,HEMATOCRIT_interval_f1,HEMATOCRIT_interval_f2,HEMATOCRIT_diff,HEMOGLOBIN_interval_f1,HEMOGLOBIN_interval_f2,HEMOGLOBIN_diff,"LACTATE, ABG_interval_f1","LACTATE, ABG_interval_f2","LACTATE, ABG_diff",MAGNESIUM_interval_f1,MAGNESIUM_interval_f2,MAGNESIUM_diff,"OXYGEN PERCENT, ABG_interval_f1","OXYGEN PERCENT, ABG_interval_f2","OXYGEN PERCENT, ABG_diff","PCO2, ABG_interval_f1","PCO2, ABG_interval_f2","PCO2, ABG_diff","PH, ABG_interval_f1","PH, ABG_interval_f2","PH, ABG_diff","PHOSPHORUS, INORGANIC_interval_f1","PHOSPHORUS, INORGANIC_interval_f2","PHOSPHORUS, INORGANIC_diff",PLATELET COUNT_interval_f1,PLATELET COUNT_interval_f2,PLATELET COUNT_diff,"PO2, ABG_interval_f1","PO2, ABG_interval_f2","PO2, ABG_diff",POTASSIUM_interval_f1,POTASSIUM_interval_f2,POTASSIUM_diff,SODIUM_interval_f1,SODIUM_interval_f2,SODIUM_diff,WBC_interval_f1,WBC_interval_f2,WBC_diff,HR_mean,O2Sat_mean,SBP_mean,MAP_mean,Resp_mean,O2Sat_median,SBP_median,MAP_median,Resp_median,O2Sat_min,SBP_min,MAP_min,Resp_min,O2Sat_max,SBP_max,MAP_max,Resp_max,O2Sat_std,SBP_std,MAP_std,Resp_std,O2Sat_dstd,SBP_dstd,MAP_dstd,Resp_dstd,HR_score,Temp_score,Resp_score,MAP_score,qsofa,SIRS,PredictedProbability,SepsisLabel_0.005,SepsisLabel_0.01,SepsisLabel_0.02,SepsisLabel_0.03,SepsisLabel_0.04,SepsisLabel_0.05,SepsisLabel_0.06,SepsisLabel_0.07,SepsisLabel_0.09,SepsisLabel_0.1,SepsisLabel_0.15,SepsisLabel_0.2,SepsisLabel_0.25,SepsisLabel_0.3,SepsisLabel_0.4,SepsisLabel_0.5,Prediction_Time
0,*Unspecified,TH 6 MAIN PERI-OP,Tisch,0.0000,0.5000,2022-08-03 06:36:00,2022-08-03 06:06:00,20220803,NaN,132.0000,76.0000,88.0000,15.0000,36.1140,"3,086.4400",33.1100,97.0000,NaN,94.6667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.1657,0.0000,1.0000,0.0000,-1.0000,NaN,1.0000,0.0000,NaN,1.0000,0.0000,NaN,1.0000,0.0000,NaN,1.0000,0.0000,NaN,1.0000,0.0000,NaN,1.0000,0.0000,NaN,1.0000,0.0000,NaN,1.0000,0.0000,NaN,0.0000,-1.0000,NaN,1.0000,0.0000,NaN,0.0000,-1.0000,NaN,0.0000,-1.0000,NaN,0.0000,-1

In [3]:
pred_df_full['ID'].nunique()

14085

In [4]:
intervention_df = pd.read_pickle('data/intervention_prospective_val_20220805_20220930_20221010.pkl')
intervention_df = intervention_df[intervention_df['ID'].isin(pred_df_full['ID'].tolist())]
intervention_df.sort_values(by=['rel_time'], inplace=True)

select_cols = ['ID', 'DischargeInstant',
 'DischargeDateKey',
 'Sepsis_ICD_List',
 'Sepsis_Case',
 'Lactate_Values',
 'Lactate_Ordered',
 'Abx_Medication_Values',
 'Abx_Ordered',
 'Vent_Values',
 'IMV',
 'Vasopressor_Medication_Values',
 'Vasopressor_Ordered',
 'Blood_Culture_Values',
 'Blood_Culture_Ordered',
 'INR_MELD_ResultInstant',
 'INR_MELD_Value']
intervention_df = intervention_df[select_cols]
intervention_df.drop_duplicates(subset=[cols for cols in intervention_df.columns.tolist() if cols not in['Abx_Medication_Values', 'Sepsis_ICD_List', 'Lactate_Values', 'Vent_Values', 'Vasopressor_Medication_Values', 'Blood_Culture_Values']], keep='first',inplace=True)
intervention_df.iloc[:, 3:]

,Sepsis_ICD_List,Sepsis_Case,Lactate_Values,Lactate_Ordered,Abx_Medication_Values,Abx_Ordered,Vent_Values,IMV,Vasopressor_Medication_Values,Vasopressor_Ordered,Blood_Culture_Values,Blood_Culture_Ordered,INR_MELD_ResultInstant,INR_MELD_Value
0,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,[Positive for MSSA (Methicillin Susceptible St...,1.0000,NaT,NaN
1339248,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,2022-09-16 00:36:00,1.10
1339247,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,"[** No MRSA or MSSA isolated., Performed by NY...",1.0000,2022-09-15 21:45:00,1.3
1339246,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaT,NaN
1339245,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,"[** No MRSA or MSSA isolated., Performed by NY...",1.0000,2022-09-15 20:23:00,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1566685,[A41.9],1.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaT,NaN
219752,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaT,NaN
245091,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaT,NaN
1669765,[A41.9],1.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaT,NaN


In [5]:
pred_df_full = pred_df_full.merge(intervention_df, on=['ID'], how='left')
pred_df_full.iloc[:, 3:]

,DepartmentServiceGrouper,DepartmentName,LocationAbbreviation,LOS,rel_time,abs_time,AdmissionInstant,AdmissionDateKey,etco2,SBP,DBP,HR,Resp,Temp,Weight,BMI,SpO2,gcs_total_score,MAP,ALKALINE PHOSPHATASE,APTT,"ARTERIAL PCO2, POC","ARTERIAL PH, POC","ARTERIAL PO2, POC",AST,BILIRUBIN TOTAL,BLOOD UREA NITROGEN,CALCIUM,CHLORIDE,CREATININE,FIBRINOGEN,"FIO2, ABG",GLUCOSE,HEMATOCRIT,HEMOGLOBIN,"LACTATE, ABG",MAGNESIUM,"OXYGEN PERCENT, ABG","PCO2, ABG","PH, ABG","PHOSPHORUS, INORGANIC",PLATELET COUNT,"PO2, ABG",POTASSIUM,SODIUM,WBC,AgeAtAdmissioninYears,sex_Female,sex_Male,etco2_interval_f1,etco2_interval_f2,etco2_diff,SBP_interval_f1,SBP_interval_f2,SBP_diff,DBP_interval_f1,DBP_interval_f2,DBP_diff,HR_interval_f1,HR_interval_f2,HR_diff,Resp_interval_f1,Resp_interval_f2,Resp_diff,Temp_interval_f1,Temp_interval_f2,Temp_diff,Weight_interval_f1,Weight_interval_f2,Weight_diff,BMI_interval_f1,BMI_interval_f2,BMI_diff,SpO2_interval_f1,SpO2_interval_f2,SpO2_diff,gcs_total_score_interval_f1,gcs_total_score_interval_f2,gcs_total_score_diff,MAP_interval_f1,MAP_interval_f2,MAP_diff,ALKALINE PHOSPHATASE_interval_f1,ALKALINE PHOSPHATASE_interval_f2,ALKALINE PHOSPHATASE_diff,APTT_interval_f1,APTT_interval_f2,APTT_diff,"ARTERIAL PCO2, POC_interval_f1","ARTERIAL PCO2, POC_interval_f2","ARTERIAL PCO2, POC_diff","ARTERIAL PH, POC_interval_f1","ARTERIAL PH, POC_interval_f2","ARTERIAL PH, POC_diff","ARTERIAL PO2, POC_interval_f1","ARTERIAL PO2, POC_interval_f2","ARTERIAL PO2, POC_diff",AST_interval_f1,AST_interval_f2,AST_diff,BILIRUBIN TOTAL_interval_f1,BILIRUBIN TOTAL_interval_f2,BILIRUBIN TOTAL_diff,BLOOD UREA NITROGEN_interval_f1,BLOOD UREA NITROGEN_interval_f2,BLOOD UREA NITROGEN_diff,CALCIUM_interval_f1,CALCIUM_interval_f2,CALCIUM_diff,CHLORIDE_interval_f1,CHLORIDE_interval_f2,CHLORIDE_diff,CREATININE_interval_f1,CREATININE_interval_f2,CREATININE_diff,"FIO2, ABG_interval_f1","FIO2, ABG_interval_f2","FIO2, ABG_diff",GLUCOSE_interval_f1,GLUCOSE_interval_f2,GLUCOSE_diff,HEMATOCRIT_interval_f1,HEMATOCRIT_interval_f2,HEMATOCRIT_diff,HEMOGLOBIN_interval_f1,HEMOGLOBIN_interval_f2,HEMOGLOBIN_diff,"LACTATE, ABG_interval_f1","LACTATE, ABG_interval_f2","LACTATE, ABG_diff",MAGNESIUM_interval_f1,MAGNESIUM_interval_f2,MAGNESIUM_diff,"OXYGEN PERCENT, ABG_interval_f1","OXYGEN PERCENT, ABG_interval_f2","OXYGEN PERCENT, ABG_diff","PCO2, ABG_interval_f1","PCO2, ABG_interval_f2","PCO2, ABG_diff","PH, ABG_interval_f1","PH, ABG_interval_f2","PH, ABG_diff","PHOSPHORUS, INORGANIC_interval_f1","PHOSPHORUS, INORGANIC_interval_f2","PHOSPHORUS, INORGANIC_diff",PLATELET COUNT_interval_f1,PLATELET COUNT_interval_f2,PLATELET COUNT_diff,"PO2, ABG_interval_f1","PO2, ABG_interval_f2","PO2, ABG_diff",POTASSIUM_interval_f1,POTASSIUM_interval_f2,POTASSIUM_diff,SODIUM_interval_f1,SODIUM_interval_f2,SODIUM_diff,WBC_interval_f1,WBC_interval_f2,WBC_diff,HR_mean,O2Sat_mean,SBP_mean,MAP_mean,Resp_mean,O2Sat_median,SBP_median,MAP_median,Resp_median,O2Sat_min,SBP_min,MAP_min,Resp_min,O2Sat_max,SBP_max,MAP_max,Resp_max,O2Sat_std,SBP_std,MAP_std,Resp_std,O2Sat_dstd,SBP_dstd,MAP_dstd,Resp_dstd,HR_score,Temp_score,Resp_score,MAP_score,qsofa,SIRS,PredictedProbability,SepsisLabel_0.005,SepsisLabel_0.01,SepsisLabel_0.02,SepsisLabel_0.03,SepsisLabel_0.04,SepsisLabel_0.05,SepsisLabel_0.06,SepsisLabel_0.07,SepsisLabel_0.09,SepsisLabel_0.1,SepsisLabel_0.15,SepsisLabel_0.2,SepsisLabel_0.25,SepsisLabel_0.3,SepsisLabel_0.4,SepsisLabel_0.5,Prediction_Time,DischargeInstant,DischargeDateKey,Sepsis_ICD_List,Sepsis_Case,Lactate_Values,Lactate_Ordered,Abx_Medication_Values,Abx_Ordered,Vent_Values,IMV,Vasopressor_Medication_Values,Vasopressor_Ordered,Blood_Culture_Values,Blood_Culture_Ordered,INR_MELD_ResultInstant,INR_MELD_Value
0,*Unspecified,TH 6 MAIN PERI-OP,Tisch,0.0000,0.5000,2022-08-03 06:36:00,2022-08-03 06:06:00,20220803,NaN,132.0000,76.0000,88.0000,15.0000,36.1140,"3,086.4400",33.1100,97.0000,NaN,94.6667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [6]:
pred_df_full.dropna(subset=['DischargeInstant'], inplace=True) #include only encounters that have been discharged.

In [7]:
pred_df_full['ID'].nunique()

13816

# II. Choose High Risk Group and Medium Risk Group

In [8]:
pred_df_full.sort_values(by=['rel_time'], inplace=True)
pos_df = pred_df_full[pred_df_full[f'SepsisLabel_0.5'] == 1]
pos_df.drop_duplicates(subset=['ID'], keep='first', inplace=True)
pos_df['Sepsis_Grouper'] = f'High Risk, ML Sepsis Score (0.5, 1.0], N={pos_df.shape[0]}'
    
neg_df = pred_df_full[~pred_df_full['ID'].isin(pos_df['ID'].tolist())] #subset of encounters who NEVER crossed 0.5 threshold.

print(f'With 0.25 as the min default threshold, find the upper threshold that produces a similar \nN={pos_df.shape[0]} as in the high risk group')
print('---' * 20)
thresholds = [0.26, 0.27, 0.28, 0.29, 0.30, 0.31, 0.32, 0.33, 0.34, 0.35]
for upper_threshold in thresholds:
    df = neg_df.query(f'0.25 <= PredictedProbability < {upper_threshold}')
    df.drop_duplicates(subset=['ID'], keep='first', inplace=True)
    print(f'min threshold: 0.25, max threshold: {upper_threshold}, {df.shape}')
    

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
<ipython-input-8-4e35d5dfe08e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_df['Sepsis_Grouper'] = f'High Risk, ML Sepsis Score (0.5, 1.0], N={pos_df.shape[0]}'


With 0.25 as the min default threshold, find the upper threshold that produces a similar 
N=1834 as in the high risk group
------------------------------------------------------------
min threshold: 0.25, max threshold: 0.26, (1257, 228)
min threshold: 0.25, max threshold: 0.27, (1536, 228)
min threshold: 0.25, max threshold: 0.28, (1637, 228)
min threshold: 0.25, max threshold: 0.29, (1704, 228)
min threshold: 0.25, max threshold: 0.3, (1797, 228)
min threshold: 0.25, max threshold: 0.31, (1844, 228)
min threshold: 0.25, max threshold: 0.32, (1934, 228)
min threshold: 0.25, max threshold: 0.33, (1965, 228)
min threshold: 0.25, max threshold: 0.34, (1987, 228)
min threshold: 0.25, max threshold: 0.35, (2002, 228)


In [9]:
#Get Medium and High Risk Group
min_threshold = 0.25
max_threshold = 0.31 #N=1844 is closest to N=1834 for the high risk group
neg_df = neg_df.query(f'{min_threshold} <= PredictedProbability < {max_threshold}')
neg_df.drop_duplicates(subset=['ID'], keep='first', inplace=True)
neg_df['Sepsis_Grouper'] = f'Medium Risk, ML Sepsis Score [0.25, 0.31), N={neg_df.shape[0]}'
at_risk_df = pd.concat([pos_df, neg_df])

#Combine with all encounters not in Medium and High Risk Group
other_encounters = pred_df_full[~pred_df_full['ID'].isin(at_risk_df['ID'].tolist())]
other_encounters.sort_values(by=['rel_time'], inplace=True)
other_encounters.drop_duplicates(subset=['ID'], keep='first', inplace=True)
other_encounters['Sepsis_Grouper'] = np.nan

final_df = pd.concat([at_risk_df, other_encounters])
final_df.iloc[:, 3:]

<ipython-input-9-61aea482deaa>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_encounters['Sepsis_Grouper'] = np.nan


,DepartmentServiceGrouper,DepartmentName,LocationAbbreviation,LOS,rel_time,abs_time,AdmissionInstant,AdmissionDateKey,etco2,SBP,DBP,HR,Resp,Temp,Weight,BMI,SpO2,gcs_total_score,MAP,ALKALINE PHOSPHATASE,APTT,"ARTERIAL PCO2, POC","ARTERIAL PH, POC","ARTERIAL PO2, POC",AST,BILIRUBIN TOTAL,BLOOD UREA NITROGEN,CALCIUM,CHLORIDE,CREATININE,FIBRINOGEN,"FIO2, ABG",GLUCOSE,HEMATOCRIT,HEMOGLOBIN,"LACTATE, ABG",MAGNESIUM,"OXYGEN PERCENT, ABG","PCO2, ABG","PH, ABG","PHOSPHORUS, INORGANIC",PLATELET COUNT,"PO2, ABG",POTASSIUM,SODIUM,WBC,AgeAtAdmissioninYears,sex_Female,sex_Male,etco2_interval_f1,etco2_interval_f2,etco2_diff,SBP_interval_f1,SBP_interval_f2,SBP_diff,DBP_interval_f1,DBP_interval_f2,DBP_diff,HR_interval_f1,HR_interval_f2,HR_diff,Resp_interval_f1,Resp_interval_f2,Resp_diff,Temp_interval_f1,Temp_interval_f2,Temp_diff,Weight_interval_f1,Weight_interval_f2,Weight_diff,BMI_interval_f1,BMI_interval_f2,BMI_diff,SpO2_interval_f1,SpO2_interval_f2,SpO2_diff,gcs_total_score_interval_f1,gcs_total_score_interval_f2,gcs_total_score_diff,MAP_interval_f1,MAP_interval_f2,MAP_diff,ALKALINE PHOSPHATASE_interval_f1,ALKALINE PHOSPHATASE_interval_f2,ALKALINE PHOSPHATASE_diff,APTT_interval_f1,APTT_interval_f2,APTT_diff,"ARTERIAL PCO2, POC_interval_f1","ARTERIAL PCO2, POC_interval_f2","ARTERIAL PCO2, POC_diff","ARTERIAL PH, POC_interval_f1","ARTERIAL PH, POC_interval_f2","ARTERIAL PH, POC_diff","ARTERIAL PO2, POC_interval_f1","ARTERIAL PO2, POC_interval_f2","ARTERIAL PO2, POC_diff",AST_interval_f1,AST_interval_f2,AST_diff,BILIRUBIN TOTAL_interval_f1,BILIRUBIN TOTAL_interval_f2,BILIRUBIN TOTAL_diff,BLOOD UREA NITROGEN_interval_f1,BLOOD UREA NITROGEN_interval_f2,BLOOD UREA NITROGEN_diff,CALCIUM_interval_f1,CALCIUM_interval_f2,CALCIUM_diff,CHLORIDE_interval_f1,CHLORIDE_interval_f2,CHLORIDE_diff,CREATININE_interval_f1,CREATININE_interval_f2,CREATININE_diff,"FIO2, ABG_interval_f1","FIO2, ABG_interval_f2","FIO2, ABG_diff",GLUCOSE_interval_f1,GLUCOSE_interval_f2,GLUCOSE_diff,HEMATOCRIT_interval_f1,HEMATOCRIT_interval_f2,HEMATOCRIT_diff,HEMOGLOBIN_interval_f1,HEMOGLOBIN_interval_f2,HEMOGLOBIN_diff,"LACTATE, ABG_interval_f1","LACTATE, ABG_interval_f2","LACTATE, ABG_diff",MAGNESIUM_interval_f1,MAGNESIUM_interval_f2,MAGNESIUM_diff,"OXYGEN PERCENT, ABG_interval_f1","OXYGEN PERCENT, ABG_interval_f2","OXYGEN PERCENT, ABG_diff","PCO2, ABG_interval_f1","PCO2, ABG_interval_f2","PCO2, ABG_diff","PH, ABG_interval_f1","PH, ABG_interval_f2","PH, ABG_diff","PHOSPHORUS, INORGANIC_interval_f1","PHOSPHORUS, INORGANIC_interval_f2","PHOSPHORUS, INORGANIC_diff",PLATELET COUNT_interval_f1,PLATELET COUNT_interval_f2,PLATELET COUNT_diff,"PO2, ABG_interval_f1","PO2, ABG_interval_f2","PO2, ABG_diff",POTASSIUM_interval_f1,POTASSIUM_interval_f2,POTASSIUM_diff,SODIUM_interval_f1,SODIUM_interval_f2,SODIUM_diff,WBC_interval_f1,WBC_interval_f2,WBC_diff,HR_mean,O2Sat_mean,SBP_mean,MAP_mean,Resp_mean,O2Sat_median,SBP_median,MAP_median,Resp_median,O2Sat_min,SBP_min,MAP_min,Resp_min,O2Sat_max,SBP_max,MAP_max,Resp_max,O2Sat_std,SBP_std,MAP_std,Resp_std,O2Sat_dstd,SBP_dstd,MAP_dstd,Resp_dstd,HR_score,Temp_score,Resp_score,MAP_score,qsofa,SIRS,PredictedProbability,SepsisLabel_0.005,SepsisLabel_0.01,SepsisLabel_0.02,SepsisLabel_0.03,SepsisLabel_0.04,SepsisLabel_0.05,SepsisLabel_0.06,SepsisLabel_0.07,SepsisLabel_0.09,SepsisLabel_0.1,SepsisLabel_0.15,SepsisLabel_0.2,SepsisLabel_0.25,SepsisLabel_0.3,SepsisLabel_0.4,SepsisLabel_0.5,Prediction_Time,DischargeInstant,DischargeDateKey,Sepsis_ICD_List,Sepsis_Case,Lactate_Values,Lactate_Ordered,Abx_Medication_Values,Abx_Ordered,Vent_Values,IMV,Vasopressor_Medication_Values,Vasopressor_Ordered,Blood_Culture_Values,Blood_Culture_Ordered,INR_MELD_ResultInstant,INR_MELD_Value,Sepsis_Grouper
284325,*Unspecified,LI NW 1 MB 1400,NYUWH,0.0000,0.5000,2022-07-24 21:13:00,2022-07-24 20:43:00,20220724,NaN,82.5000,50.0000,91.0000,21.0000,36.7252,"1,760.0000",23.8500,100.0000,NaN,61.1667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [10]:
final_df.to_csv('data/main_prospective_val_df_20220805_20220930_20221010.csv')